# P06: Entropy Production & Thermodynamic Uncertainty Relations (PhD Level)

## 1. Introduction: The Cost of Flow

Equilibrium systems (M2, M3) are dead: there is no net current. Living cities, like living cells, are Non-Equilibrium Steady States (NESS). They maintain a constant cycle of flow (commute, logistics, money) at the cost of dissipating energy.

How do we quantify "how far" a city is from equilibrium? The answer is **Entropy Production Rate (EPR)**, $\sigma$.

In this PhD-level notebook, we will:
1. Construct a minimal NESS: A biased 3-state loop.
2. Calculate $\sigma$ explicitly from the breakage of Detailed Balance.
3. Verify the **Thermodynamic Uncertainty Relation (TUR)** ($Q \ge 2k_B$), a profound recent (2015) discovery connecting **Dissipation** ($\sigma$) with **Precision** ($1/\epsilon^2$). It tells us: "You cannot have a precise clock (or punctual metro system) without burning entropy."

### Core Mappings
| Driven Urban System | Non-Equilibrium Physics |
| :--- | :--- |
| Directed Flow (e.g. Ring Road) | Net Current $J$ |
| Asymmetric Rates ($W_{ij} \neq W_{ji}$) | Broken Detailed Balance |
| Driving Force (Policy/Potential) | Affinity $A$ |
| "Inefficiency" / Waste | Entropy Production $\sigma$ |
| Reliability (Low variance) | Precision $J^2 / \text{Var}(J)$ |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

# Set seed
np.random.seed(42)

## 2. The Minimal NESS Model (3-State Ring)

Consider 3 nodes (Regions A, B, C) in a ring.
Rates:
- Clockwise ($k_+$): $A \to B \to C \to A$
- Counter-Clockwise ($k_-$): $A \leftarrow B \leftarrow C \leftarrow A$

If $k_+ \neq k_-$, Detailed Balance is broken. There is a net current $J = P_{ss}(k_+ - k_-)$.

In [ ]:
# Parameters
k_plus = 10.0  # Driven forward rate
k_minus = 1.0  # Backward rate

# Transition Rate Matrix W (W_ij is j -> i, verify convention!)
# Usually Master Eq: dP/dt = W P
# Here we define W[i, j] as rate FROM j TO i
W = np.zeros((3, 3))

# 0->1, 1->2, 2->0 (Clockwise)
W[1, 0] = k_plus
W[2, 1] = k_plus
W[0, 2] = k_plus

# 0<-1, 1<-2, 2<-0 (Counter-Clockwise)
W[0, 1] = k_minus
W[1, 2] = k_minus
W[2, 0] = k_minus

# Diagonal elements reflect outflow (sum of col must be 0)
for i in range(3):
    W[i, i] = -np.sum(W[:, i])

print("Rate Matrix W:\n", W)

## 3. Entropy Production Rate (EPR)

The definition of EPR (entropy produced per unit time) is:
$$ \sigma = \sum_{i,j} P_i W_{ji} \ln \frac{P_i W_{ji}}{P_j W_{ij}} $$
For our symmetric ring (uniform steady state $P_i = 1/3$), this simplifies to:
$$ \sigma = (J_+ + J_-) \ln \frac{k_+}{k_-} = \dots $$
Actually, net flux $J_{net} = P(k_+ - k_-)$. 
The Force (Affinity) in the loop is $A = \ln(k_+^3 / k_-^3) = 3 \ln(k_+/k_-)$.
So $\sigma_{total} = J_{net} \times A$.

In [ ]:
# 1. Steady State
# Solve W @ P = 0. 
# Since rates are symmetric per node type, P = [1/3, 1/3, 1/3]
P_ss = np.array([1/3, 1/3, 1/3])

# 2. Calculate Flux J
J_cw = P_ss[0] * k_plus
J_ccw = P_ss[0] * k_minus
J_net = J_cw - J_ccw

# 3. Calculate EPR (sigma)
# Sum over all links. There are 3 links.
# Contribution per link: P_i * W_ji * ln(...) + P_j * W_ij * ln(...)
# It simplifies to: (J_cw - J_ccw) * ln(J_cw / J_ccw)
sigma_per_link = (J_cw - J_ccw) * np.log(J_cw / J_ccw)
sigma_total = 3 * sigma_per_link

print(f"Net Current J: {J_net:.4f}")
print(f"Entropy Production Rate sigma: {sigma_total:.4f}")
print("-> Positive sigma confirms Irreversibility.")

## 4. Thermodynamic Uncertainty Relation (TUR)

Why does physics care about $\sigma$? Because it bounds Precision.
TUR (Barato & Seifert, 2015) states:
$$ \frac{\text{Var}(J)}{J^2} \ge \frac{2}{\sigma \tau} $$
Or: $\mathcal{Q} = \epsilon^2 \cdot (\sigma \tau) \ge 2$.

This means: To have a very steady current (low relative variance $\epsilon^2$), you MUST pay a high cost in entropy $\sigma$.

Let's simulate the process using Gillespie algorithm to measure $\text{Var}(J)$ and verify this bound.

In [ ]:
def gillespie_flux(steps, k_plus, k_minus):
    # Standard Gillespie, but we track the net displacement (flux) theta
    theta = 0 
    rate_sum = k_plus + k_minus # Total rate out of any state (symmetric)
    
    # Time increment is exponential
    times = np.random.exponential(1/rate_sum, size=steps)
    total_time = np.sum(times)
    
    # Jumps: +1 with prob k+ / sum, -1 with prob k- / sum
    # We don't even need to track state i, because the loop is homogeneous!
    p_plus = k_plus / rate_sum
    jumps = np.where(np.random.rand(steps) < p_plus, 1, -1)
    
    net_displacement = np.sum(jumps)
    # Flux J = Displacement / Time
    J_obs = net_displacement / total_time
    return J_obs, total_time

# Run many trajectories to estimate Variance of J
n_trials = 1000
steps_per_trial = 500
J_samples = []
tau_samples = []

for _ in range(n_trials):
    j, t = gillespie_flux(steps_per_trial, k_plus, k_minus)
    J_samples.append(j)
    tau_samples.append(t)

J_mean = np.mean(J_samples)
J_var = np.var(J_samples)
tau_mean = np.mean(tau_samples)

print(f"Simulated Mean J: {J_mean:.4f} (Theory: {J_net:.4f})")
print(f"Simulated Var(J): {J_var:.4f}")
print(f"Observation Time tau: {tau_mean:.4f}")

# Check TUR
# Product Q = epsilon^2 * (sigma * tau)
epsilon_sq = J_var / (J_mean**2)
dissipation = sigma_total * tau_mean
Q = epsilon_sq * dissipation

print(f"\nTUR Product Q: {Q:.4f}")
print(f"Is Q >= 2? {Q >= 2.0}")
if Q >= 2.0:
    print("✅ Theoretical bound holds!")
else:
    print("❌ Bound violated (check definitions).")